In [3]:
!pip install scipy
!pip install Bio
!pip install xmltramp2

In [4]:
import re
import pickle
import numpy as np
from scipy.spatial.distance import hamming
from Bio import pairwise2
from Bio.Seq import Seq

## Data Importation:

In [5]:
'''
    Description: This data structure is used to save insulin gene sequence and their relative info.
    Attributes :
        id: the gene id of the insulin gene given by NCBI dataset
        name: the organism of the gene sequence
        seq: the insulin gene sequence as Bio.Seq 
'''
class gene:
    def __init__(self, name, id, sequence):
        self.id = id  
        self.name = name      
        self.seq = None
        self.str = sequence

In [6]:
'''
    Description: this function will read the NCBI dataset file in the given path
                 and return all gene as a list of gene data structure
'''
def data_reader(path):
    gene_dict = {}
    f = open(path, "r")
    line = f.readline()
    gene_temp = None
    while line != "":
        line = line.replace('\n', '')
        if line[0] == '>':
            info = re.split(r"\[|\]|=",line)
            if gene_dict.get(info[2]) is None:
                gene_temp = gene(info[2],info[5],"")
                gene_dict[info[2]] = gene_temp
        else:
            gene_temp.str = gene_temp.str + line
        line = f.readline()
    f.close()
    for i in gene_dict.keys():
        gene_dict[i].bio = Seq(gene_dict[i].str)
    return gene_dict

def data_saver(data):
    afile = open(r'./SavedData.pkl', 'wb')
    pickle.dump(data, afile)
    afile.close()

def data_loader():
    file2 = open(r'./SavedData.pkl', 'rb')
    new_d = pickle.load(file2)
    file2.close()
    return new_d

In [7]:
path = "../ncbi_dataset/data/gene.fna"
candidate = ["Homo sapiens", "Macaca mulatta", "Macaca fascicularis", "Gorilla gorilla", "Macaca nemestrina", "Sus scrofa", "Peromyscus leucopus", "Peromyscus maniculatus bairdii"]
gene_dict = data_reader(path)
print(len(gene_dict))

8


## Multisequence Alignment (From HW2)

In [8]:
def compute_profile(alignment, alphabet):
    """
    Given an alphabet an a multiple sequence alignment in that alphabet,
    computes and returns its profile representation
    
    :param: alignment is a list of lists of characters in the alphabet
    :param: alphabet is a list of characters in the alphabet from which the strings are
            constructed
    :return: a dictionary where dict[x][i] is the frequency of the character
             x in the i-th position of the alignment.
    """
    
    if not alignment:
        return {}

    n = len(alignment)
    l = len(alignment[0])
    profile = {}
    
    # YOUR CODE HERE
    for alpha in alphabet:
      profile[alpha] = [0.0]*l
    for gene in alignment:
      for c in range(l):
        profile[gene[c]][c] += 1
    for i in profile.keys():
      for j in range(len(profile[i])):
        profile[i][j] /= n

    return profile

def compute_tau(profile, alphabet, delta):
    """
    Given a profile, an alphabet and a scoring function for that alphabet,
    returns the scoring function for aligning a character in the alphabet
    to a column in the profile
    
    :param: profile is the profile representation of the multiple sequence
            we are aligning against
    :param: alphabet is the alphabet of characters that compose our sequences
    :param: delta is the scoring function between characters in our alphabet
    
    :return: The scoring function tau such that tau[x][i] is the score for aligning
             character x with column i of the profile.
    """
    tau = {}
    n = len(profile[alphabet[0]])
    m = len(profile)
    # YOUR CODE HERE
    for alpha in alphabet:
      tau[alpha] = [0.0]*n
    for x in alphabet:
      for j in range(n):
        for y in alphabet:
          tau[x][j] += delta[x][y]*profile[y][j]
    return tau

UP = (-1, 0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)

def traceback_1(aln1, aln2, pointers):
    i = len(aln1[0])-1
    j = len(aln2[0])-1
    new_al1 = [list(v) for v in aln1]
    new_al2 = [list(w) for w in aln2]
    while True:
        di, dj = pointers[i][j]
        if (di, dj) == LEFT:
            for seq1 in new_al1:
                seq1.insert(i, '-')
        if (di, dj) == UP:
            for seq2 in new_al2:
                seq2.insert(j, '-')
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    new_alignment = []
    for seq in new_al1:
        new_alignment.append(''.join(seq))
    for seq in new_al2:
        new_alignment.append(''.join(seq))
    return new_alignment


def align_sequence_profile(alignment, sequence, alphabet, delta):
    """
    This function aligns a sequence against a multiple sequence alignment
    
    :param: alignment is the multiple sequence alignment are aligning against.
            This is a list of list of characters
    :param: sequence is the new sequence we are aligning to the multiple alignment.
            This is a list of characters
    :param: alphabet is a list of characters that could compose the sequences in
            the alignments.
    :param: delta is the scoring function for aligning characters in our alphabet.
            delta[x][y] is the score for aligning the characters x and y.
    
    
    :return: a list of lists of characters in the alphabet, representing the 
             new multiple sequence alignment
    """
    # Base case when there is an empty multiple alignment
    if not alignment:
        return [sequence]
    M = [[0 for _ in range(len(alignment[0]))] for _ in range(len(sequence))] 
    pointers = [[(0,0) for _ in range(len(alignment[0]))] for _ in range(len(sequence))]
    score = None
    
    profile = compute_profile(alignment, alphabet)
    tau = compute_tau(profile, alphabet, delta)

    for i in range(len(sequence)):
        for j in range(len(alignment[0])):
            if i == 0 and j == 0:   
                M[i][j] = 0
            elif i == 0:
                M[i][j] = M[i][j-1] + tau['-'][j-1]
                pointers[i][j] = LEFT
            elif j == 0:
                sequence[i-1]
                M[i][j] = M[i-1][j] + delta[sequence[i-1]]['-']
                pointers[i][j] = UP
            else:
                best_sub = max([(LEFT, M[i][j-1] + tau['-'][j-1]), 
                               (UP, M[i-1][j] + delta[sequence[i-1]]['-']), 
                               (TOPLEFT, M[i-1][j-1] + tau[sequence[i-1]][j-1])], key = lambda x: x[1])
                pointers[i][j] = best_sub[0]
                M[i][j] = best_sub[1]

    score = M[-1][-1]
    return score, traceback_1([sequence], alignment, pointers)

def compute_sigma(p,q,alphabet, delta):
    """
    :param: p is the profile for the first multiple alignment
    :param: q is the profile for the second multiple alignment
    :param: alphabet is the list of all characters in our sequences
    :param: delta is the scoring function for aligning characters in our alphabet
    
    :returns: a list of lists sigma such that sigma[i][j] is the score for aligning column
              i of p with column j of q
    """
    sigma = []
    size_p = len(p[alphabet[0]])
    size_q = len(q[alphabet[0]])
    # YOUR CODE HERE
    for i in range(size_p):
      sigma.append([])
      for j in range(size_q):
        sigma[i].append(0)
        for x in alphabet:
          for y in alphabet:
            sigma[i][j] += p[x][i]*q[y][j]*delta[x][y]
    return sigma

def traceback_2(aln1, aln2, pointers):
    i = len(aln1[0])
    j = len(aln2[0])
    new_al1 = [list(v) for v in aln1]
    new_al2 = [list(w) for w in aln2]
    while True:
        di, dj = pointers[i][j]
        if (di, dj) == LEFT:
            for seq1 in new_al1:
                seq1.insert(i, '-')
        if (di, dj) == UP:
            for seq2 in new_al2:
                seq2.insert(j, '-')
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    new_alignment = []
    for seq in new_al1:
        new_alignment.append(''.join(seq))
    for seq in new_al2:
        new_alignment.append(''.join(seq))
    return new_alignment

def align_profile_profile(aln1, aln2, alphabet, delta):
    """
    :param: aln1 is a list of lists representing the first multiple alignment
    :param: aln2 is a list of lists representing the second multiple alignment
    :param: alphabet is the alphabet from which the sequences are derived
    :param: delta is a scoring function. delta(x,y) gives us the score for aligning 
            character x with character y in our alphabet
            
    :returns: the optimal score and the optimal multiple alignment for the two input alignments.
    """
    # Base case when there is an empty multiple alignment
    if not aln1 and not aln2:
        return []
    elif not aln2:
        return aln1
    elif not aln1:
        return aln2
    
    S = [[0 for j in range(len(aln2[0])+1)] for i in range(len(aln1[0])+1)] 
    pointers = [[(0,0) for j in range(len(aln2[0])+1)] for i in range(len(aln1[0])+1)]
    score = None
    
    # Compute profiles and scoring functions here
    p = compute_profile(aln1, alphabet)
    q = compute_profile(aln2, alphabet)
    tau1 = compute_tau(p, alphabet, delta)
    tau2 = compute_tau(q, alphabet, delta)
    sigma = compute_sigma(p,q,alphabet, delta)
    # YOUR CODE HERE

    for j in range(0, len(aln2[0])+1):
      for i in range(0, len(aln1[0])+1):
        vals = []
        dirs = []
        if i==0 and j==0:
          vals.append(0)
          dirs.append(TOPLEFT)
        if j > 0:
          vals.append(S[i][j-1]+tau2["-"][j-1])
          dirs.append(LEFT)
        if i > 0:
          vals.append(S[i-1][j]+tau1["-"][i-1])
          dirs.append(UP)
        if i > 0 and j > 0:
          vals.append(S[i-1][j-1]+sigma[i-1][j-1])
          dirs.append(TOPLEFT)
        
        val =  max(vals)
        inx =  vals.index(val)
        S[i][j]= val
        pointers[i][j]  = dirs[inx]
    score = S[-1][-1]
    return score, traceback_2(aln1, aln2, pointers)

def greedy_progressive_align(alignments, alphabet, delta):
    """
    :param: alignments is a list of list of strings representing the sequences to be aligned
            Note: This is because we need to represent our single sequences as multiple alignments
            ,and multiple alignments are lists of strings
    :param: alphabet is the alphabet from which the sequences are derived
    :param: delta is a scoring function. delta(x,y) gives us the score for aligning 
            character x with character y in our alphabet
            
    :returns: the greedy optimal multiple sequence alignment for a given set of sequences, and the score for that alignment
    """
    
    
    while True: 
        # Base case (When to exit the loop?)
        # YOUR CODE HERE
        if len(alignments) == 1:
          return (best_score, alignments[0])
        if len(alignments) < 1:
          return None
        
        # Data structures for this iteration
        best_score = -float("inf")
        best_alignment = None
        best_m = -1
        best_n = -1

        # Compute pairwise distances 
        for m in range(len(alignments)):
            for n in range(m):
                # YOUR CODE HERE
                score, align = align_profile_profile(alignments[m], alignments[n], alphabet, delta)
                if score > best_score:
                  best_score = score
                  best_alignment = align
                  best_m = m
                  best_n = n

        # Populate the list of alignments to use for the next iteration
        next_alignments = [best_alignment]
        for i in range(len(alignments)):
            if i!=best_m and i!=best_n:
                next_alignments.append(alignments[i])
        alignments = next_alignments


In [9]:
alphabet = ['A', 'C', 'G', 'T', '-']
delta = {}
for i in range(len(alphabet)):
    delta[alphabet[i]] = {k : v for (k,v) 
                          in zip(alphabet, [1 if alphabet[i] == alphabet[j]  else -1 
                                  for j in range(len(alphabet))]
                         )}

In [10]:
## test the multialign functionality
lst = [["ACTG"],["ATTG"],["AATG"],["ATG"]]
greedy_progressive_align(lst, alphabet, delta)

(2.0, ['A-TG', 'AATG', 'ATTG', 'ACTG'])

In [13]:
def quartet_tree_generator(dict, alphabet, delta):
    quartet_trees = []    
    keys = list(dict.keys())
    n = len(keys)
    for i in range(n-3):
        for j in range(i+1,n-2):
            for k in range(j+1,n-1):
                for r in range(k+1, n):
                    compare_lst = {dict[keys[o]].str:keys[o] for o in [i,j,k,r]}
                    gene_lst = [[dict[keys[o]].str] for o in [i,j,k,r]]
                    result = greedy_progressive_align(gene_lst, alphabet, delta)
                    alignments = result[1]
                    orig = []
                    for s in alignments:
                        orig_seq = "".join(s.split("-"))
                        orig_seq_name = compare_lst.get(orig_seq)
                        if (orig_seq_name==None):
                            print("ERROR: Name not found")
                            exit(1)
                        orig.append(orig_seq_name)
                    quartet = split_quart(result[1], orig)
                    quartet_trees.append(quartet.copy())
    data_saver(quartet_trees)
    return quartet_trees
                    
def split_quart(data, orig):
    # Quartet tree representation: "ab | cd" == ((a,b), (c,d))
    seq1 = list(data[0])
    seq2 = list(data[1])
    seq3 = list(data[2])
    seq4 = list(data[3])
    lamb_12 = hamming(seq1,seq2)
    lamb_13 = hamming(seq1,seq3)
    lamb_14 = hamming(seq1,seq4)
    lamb_23 = hamming(seq2,seq3)
    lamb_24 = hamming(seq2,seq4)
    lamb_34 = hamming(seq3,seq4)
    ## find the smallest pairwise sum, 
    ## which does not include the weight of edges incident with the internal node
    path_12_34 = lamb_12 + lamb_34
    path_13_24 = lamb_13 + lamb_24
    path_14_23 = lamb_14 + lamb_23
    out = [[[orig[0],orig[1]],[orig[2],orig[3]]], [[orig[0],orig[2]],[orig[1],orig[3]]], [[orig[0],orig[3]],[orig[1],orig[2]]]]
    if path_12_34 > path_13_24: 
        temp = out[0]
    if path_13_24 > path_14_23:
        temp = out[1]
    else: 
        temp = out[2]
    sib1 = sorted(temp[0])
    sib2 = sorted(temp[1])
    print([tuple(sib1), tuple(sib2)]) 
    return [tuple(sib1), tuple(sib2)]

In [12]:
dict = gene_dict.copy()
quartet_trees = quartet_tree_generator(dict, alphabet, delta)

[('Gorilla gorilla', 'Macaca mulatta'), ('Homo sapiens', 'Macaca fascicularis')]
[('Homo sapiens', 'Macaca fascicularis'), ('Macaca mulatta', 'Macaca nemestrina')]
[('Bos taurus', 'Macaca fascicularis'), ('Homo sapiens', 'Macaca mulatta')]
[('Macaca fascicularis', 'Sus scrofa'), ('Homo sapiens', 'Macaca mulatta')]
[('Macaca fascicularis', 'Peromyscus leucopus'), ('Homo sapiens', 'Macaca mulatta')]
[('Gorilla gorilla', 'Macaca mulatta'), ('Homo sapiens', 'Macaca nemestrina')]
[('Bos taurus', 'Gorilla gorilla'), ('Homo sapiens', 'Macaca mulatta')]
[('Gorilla gorilla', 'Sus scrofa'), ('Homo sapiens', 'Macaca mulatta')]
[('Gorilla gorilla', 'Peromyscus leucopus'), ('Homo sapiens', 'Macaca mulatta')]
[('Bos taurus', 'Macaca nemestrina'), ('Homo sapiens', 'Macaca mulatta')]
[('Macaca nemestrina', 'Sus scrofa'), ('Homo sapiens', 'Macaca mulatta')]
[('Macaca nemestrina', 'Peromyscus leucopus'), ('Homo sapiens', 'Macaca mulatta')]
[('Bos taurus', 'Macaca mulatta'), ('Homo sapiens', 'Sus scrofa'

In [20]:
def removeAll(lst, elem):
    for i in range(len(lst)):
        for j in lst[i]:
            if elem in j:
                lst[i] = None
                break
    return [m for m in lst if m!=None]

def tree_assemble(quartet_trees):
    a = quartet_trees.copy()
    sib_lst = []
    while(len(a)>1):
        curr_dict = {}
        for quartet in a:
            for sib in quartet:
                if curr_dict.get(sib) is None:
                    curr_dict[sib] = 0
                curr_dict[sib] += 1
        new_sib = max(curr_dict, key=curr_dict.get)
        sib_lst.append(new_sib)
        a = removeAll(a, new_sib[0])
    return sib_lst, a

In [22]:
sib_lst, tree_top = tree_assemble(quartet_trees)
print(sib_lst)
print(tree_top)

[('Gorilla gorilla', 'Sus scrofa'), ('Macaca fascicularis', 'Sus scrofa'), ('Homo sapiens', 'Sus scrofa'), ('Macaca mulatta', 'Sus scrofa')]
[[('Macaca nemestrina', 'Sus scrofa'), ('Bos taurus', 'Peromyscus leucopus')]]
